# Deep Reinforcement Learning

> Add blockquote



 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, an platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [ ]:
!pip install gym[atari,accept-rom-license] -qq
!pip install imageio -qq

In [ ]:
import cv2
import gym
import imageio
import numpy as np
from gym import spaces

We use `gym.make()` to make an instance of a certain environemtn. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [ ]:
env_name = 'CartPole-v1'

# Create an instance of the environment
env = gym.make(env_name)

env.reset()

frames = []

for _ in range(500):
    action = env.action_space.sample()

    obs, reward, done, _ = env.step(action)

    # render this frame and add to the list of frames
    render_output = env.render(mode='rgb_array') # Change render mode to 'rgb_array'
    if render_output is not None:
        frames.append(render_output)

    if done:
        env.reset()

env.close()
imageio.mimsave('./cartpole.mp4', frames, fps=25) # Save the video file

In [ ]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

import numpy as np
import gym
from collections import deque
from tqdm import tqdm

## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.


In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, input_size, hidden_size, num_outputs):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # Fully connected layer 1
        self.fc_actor = nn.Linear(hidden_size, num_outputs) # Actor layer
        self.fc_critic = nn.Linear(hidden_size, 1) # Critic layer

    def forward(self, x):
        x = F.relu(self.fc1(x)) # Apply ReLU activation to the hidden layer
        action_probs = F.softmax(self.fc_actor(x), dim=-1) # Softmax to get action probabilities
        value = self.fc_critic(x) # Output value estimation for the state
        return action_probs, value

## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [ ]:
class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, hidden_size=256):
        self.env = env
        self.num_episodes = num_episodes
        self.max_steps = max_steps
        self.gamma = gamma
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Get the size of the observation space and the number of actions
        input_size = env.observation_space.shape[0]
        num_outputs = env.action_space.n

        # Define your actor-critic network
        self.policy_net = ActorCritic(input_size, hidden_size, num_outputs).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.critic_loss = nn.MSELoss()

    def choose_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        action_probs, _ = self.policy_net(state)
        action = torch.multinomial(action_probs, 1).item()
        return action

    def compute_returns(self, rewards):
        returns = []
        R = 0
        for r in reversed(rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns).to(self.device)
        return returns

    def train(self):
        episode_rewards = []

        for episode in tqdm(range(self.num_episodes)):
            state = self.env.reset()
            if state is None or len(state) != self.env.observation_space.shape[0]:
                print(f"Invalid initial state: {state}")
                continue

            log_probs = []
            values = []
            rewards = []
            episode_reward = 0

            for step in range(self.max_steps):
                action = self.choose_action(state)
                next_state, reward, done, _ = self.env.step(action)

                if next_state is None or len(next_state) != self.env.observation_space.shape[0]:
                    print(f"Invalid next state at step {step}: {next_state}")
                    break

                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
                _, value = self.policy_net(state_tensor)
                log_prob = torch.log(self.policy_net(state_tensor)[0][0][action])

                log_probs.append(log_prob)
                values.append(value)
                rewards.append(reward)
                episode_reward += reward

                state = next_state
                if done:
                    break

            episode_rewards.append(episode_reward)

            # Calculate the discounted rewards
            returns = self.compute_returns(rewards)

            # Convert lists to tensors
            log_probs = torch.stack(log_probs)
            values = torch.stack(values).squeeze()

            # Calculate advantage
            advantage = returns - values

            # Compute actor and critic loss
            actor_loss = -(log_probs * advantage.detach()).mean()
            critic_loss = self.critic_loss(values, returns)
            loss = actor_loss + critic_loss

            # Perform backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            print(f"Episode {episode + 1}, Reward: {episode_reward}")

        self.env.close()
        return episode_rewards

Define the model and set hyperparameters.

In [ ]:
env = gym.make('CartPole-v1')
num_episodes = 1000
max_steps = 500
lr = 1e-3
hidden_size = 256

a2c_model = A2CAgent(env, num_episodes=num_episodes, max_steps=max_steps, lr=lr, hidden_size=hidden_size)

Train the model.

In [ ]:
rewards = a2c_model.train()

## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [ ]:
env_name = 'CartPole-v1'
env = gym.make(env_name, render_mode='rgb_array')

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming a2c_model is already trained and available
model = a2c_model

frames = []
episode_rewards = []

for i in range(num_episodes):
    state = env.reset()
    if isinstance(state, tuple):  # Handle different Gym API versions
        state = state[0]

    episode_reward = 0
    done = False

    while not done:
        # Add the current frame to the list if it's the first episode
        if i == 0:
            frame = env.render()

            # Check if frame is a list and handle accordingly
            if isinstance(frame, list):
                frame = np.array(frame[0])  # Assuming the first element of the list is the frame

            # Check if the frame has the expected number of channels
            if frame.shape[-1] not in (1, 2, 3, 4):
                # Convert the frame to RGB if it has a different number of channels
                frame = frame[..., :3]  # Assuming the first 3 channels are RGB
            frames.append(frame)

        # Debugging information
        print(f"State shape: {state.shape}, State type: {type(state)}")

        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        with torch.no_grad():
            action_probs, _ = model.policy_net(state)
        action = torch.multinomial(action_probs, 1).item()

        next_state, reward, done, _ = env.step(action)
        if isinstance(next_state, tuple):  # Handle different Gym API versions
            next_state = next_state[0]

        episode_reward += reward
        state = next_state

    episode_rewards.append(episode_reward)
    print(f"Episode {i + 1} Reward: {episode_reward}")

env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")
imageio.mimsave('./test.mp4', frames, fps=25)

In [ ]:
show_video('./test.mp4')